<a href="https://colab.research.google.com/github/2303a51603/Reinforcement-learning-/blob/main/Lab%20Ass%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# Define MDP
states = [(i, j) for i in range(3) for j in range(3)]
actions = ["U", "D", "L", "R"]

goal = (2, 2)
trap = (0, 2)

def step(state, action):
    i, j = state
    if state == goal or state == trap:
        return state, 0  # terminal states

    if action == "U":
        i = max(i - 1, 0)
    elif action == "D":
        i = min(i + 1, 2)
    elif action == "L":
        j = max(j - 1, 0)
    elif action == "R":
        j = min(j + 1, 2)

    new_state = (i, j)
    if new_state == goal:
        return new_state, 1
    elif new_state == trap:
        return new_state, -1
    else:
        return new_state, 0

# Value Iteration
def value_iteration(gamma=0.9, theta=1e-4):
    V = {s: 0 for s in states}
    while True:
        delta = 0
        for s in states:
            if s in [goal, trap]:
                continue
            v = V[s]
            V[s] = max(
                sum([1.0 * (r + gamma * V[s_next])])
                for a in actions
                for s_next, r in [step(s, a)]
            )
            delta = max(delta, abs(v - V[s]))
        if delta < theta:
            break
    # extract policy
    policy = {}
    for s in states:
        if s in [goal, trap]:
            policy[s] = None
        else:
            policy[s] = max(actions, key=lambda a: step(s, a)[1] + gamma * V[step(s, a)[0]])
    return V, policy

# Policy Iteration
def policy_iteration(gamma=0.9, theta=1e-4):
    policy = {s: np.random.choice(actions) for s in states if s not in [goal, trap]}
    V = {s: 0 for s in states}

    while True:
        # Policy Evaluation
        while True:
            delta = 0
            for s in states:
                if s in [goal, trap]:
                    continue
                v = V[s]
                a = policy[s]
                s_next, r = step(s, a)
                V[s] = r + gamma * V[s_next]
                delta = max(delta, abs(v - V[s]))
            if delta < theta:
                break

        # Policy Improvement
        stable = True
        for s in states:
            if s in [goal, trap]:
                continue
            old_action = policy[s]
            policy[s] = max(actions, key=lambda a: step(s, a)[1] + gamma * V[step(s, a)[0]])
            if old_action != policy[s]:
                stable = False
        if stable:
            break

    return V, policy


# Run both algorithms
V_vi, pi_vi = value_iteration()
V_pi, pi_pi = policy_iteration()

print("Value Iteration Policy:")
print(pi_vi)
print("\nPolicy Iteration Policy:")
print(pi_pi)

Value Iteration Policy:
{(0, 0): 'D', (0, 1): 'D', (0, 2): None, (1, 0): 'D', (1, 1): 'D', (1, 2): 'D', (2, 0): 'R', (2, 1): 'R', (2, 2): None}

Policy Iteration Policy:
{(0, 0): 'D', (0, 1): 'D', (1, 0): 'D', (1, 1): 'D', (1, 2): 'D', (2, 0): 'R', (2, 1): 'R'}
